# DFT: The LDA kernel
## I. Theory

Previously we described the DFT Fock matrix as
$$F^{DFT}_{\mu\nu} = H_{\mu\nu} + 2J[D]_{\mu\nu} - \zeta K[D]_{\mu\nu} + V^{\rm{xc}}_{\mu\nu}$$
upon examination it is revealed that the only quantities that we cannot yet compute is $V^{\rm{xc}}$. 

Here we will explore the local density approximation (LDA) functionals where $V^{\rm{xc}} = f[\rho(\hat{r})]$. For these functionals the only required bit of information is the density at the grid point. As we discussed the grid last chapter we will now focus on how exactly to obtain the density on the grid.

Before we begin we should first recall that the Fock matrix is the derivative of the energy with respect atomic orbitals. Therefore, $V_{\rm{xc}}$ matrix is not the XC energy, but the derivate of that energy which can expressed as $\frac{\partial e_{\rm{xc}}}{\partial\rho}$. 

In [1]:
import psi4
import numpy as np

mol = psi4.geometry("""
He
symmetry c1
""")
psi4.set_options({'BASIS':               'CC-PVDZ',
                  'DFT_SPHERICAL_POINTS': 6,
                  'DFT_RADIAL_POINTS':    5})

svwn_w, wfn = psi4.energy("SVWN", return_wfn=True)
Vpot = wfn.V_potential()

## 2. Density on a Grid
The density on the grid can be expressed as
$$\rho(\hat{r}) = \sum\limits_{\mu\nu} D_{\mu\nu}\;\phi_\mu(\hat{r})\phi_\nu(\hat{r})$$

Recall that we compute DFT quanties on a grid, so $\hat{r}$ will run over a grid instead of all space. Using this we can build collocation matrices that map between atomic orbital and grid space $$\phi_\mu(\hat{r}) \rightarrow \phi_\mu^p$$
where our $p$ index will be the index of individual grid points. Our full expression becomes:

$$\rho_p = \phi_\mu^p D_{\mu\nu} \phi_\nu^p$$

To compute these quantities let us first remember that the DFT grid is blocked loosely over atoms. It should now be apparent to why we do this, consider the $\phi_\mu^p$ objects. The total size of this object would be nbf x npoints. To put this in perspective a moderate size molecule could have 1e4 basis functions and 1e8 grid points, so about 8 terabytes of data! As this object is very sparse it is much more convenient to store the grid and compute $\phi\mu^p$ matrices on the fly. 

We then need object to compute $\phi_\mu^p$. 

In [29]:
# Grab a "points function" to compute the Phi matrices
points_func = Vpot.properties()[0]

# Grab a block and obtain its local mapping
block = Vpot.get_block(1)
npoints = block.npoints()
lpos = np.array(block.functions_local_to_global())
print("Local basis function mapping")
print(lpos)

# Copmute phi, note the number of points and function per phi changes.
phi = np.array(points_func.basis_values()["PHI"])[:npoints, :lpos.shape[0]]
print("\nPhi Matrix")
print(phi)

Local basis function mapping
[0 1 2 3 4]

Phi Matrix
[[ 0.       0.00002  0.       0.       0.     ]
 [ 0.       0.00002  0.       0.       0.     ]
 [ 0.       0.00002  0.       0.      -0.     ]
 [ 0.       0.00002  0.       0.       0.     ]
 [ 0.       0.00002 -0.       0.       0.     ]
 [ 0.00017  0.03952  0.       0.00102  0.     ]
 [ 0.00017  0.03952  0.       0.       0.00102]
 [ 0.00017  0.03952  0.       0.      -0.00102]
 [ 0.00017  0.03952  0.00102  0.       0.     ]
 [ 0.00017  0.03952 -0.00102  0.       0.     ]
 [ 0.18842  0.21172  0.       0.52956  0.     ]
 [ 0.18842  0.21172  0.       0.       0.52956]
 [ 0.18842  0.21172  0.       0.      -0.52956]
 [ 0.18842  0.21172  0.52956  0.       0.     ]
 [ 0.18842  0.21172 -0.52956  0.       0.     ]
 [ 1.07233  0.28068  0.       0.48524  0.     ]
 [ 1.07233  0.28068  0.       0.       0.48524]
 [ 1.07233  0.28068  0.       0.      -0.48524]
 [ 1.07233  0.28068  0.48524  0.       0.     ]
 [ 1.07233  0.28068 -0.48524  0.   

## 3. Evaluating the kernel

After building the density on the grid we can then compute the exchange-correlation $f_{xc}$ at every gridpoint. This then need to be reintegrated back to atomic orbital space which can be accomplished like so:

$$V^{\rm{xc}}_{pq}[D_{pq}] = \phi_\mu^a\;\phi_\nu^a\;\; w^a\;f^a_{\rm{xc}}{(\phi_\mu^p D_{\mu\nu} \phi_\nu^p)}$$

Where $w^a$ is our combined Truetler and Lebedev weight at every point.

Unlike SCF theory where the SCF energy can be computed as the sum of the Fock and Density matrices the energy for XC kernels must be computed in grid space. Fortunately the energy is simply defined as:

$$e_{\rm{xc}} = w^a f^a_{\rm{xc}}$$

We can now put all the pieces together to compute $e_{\rm{xc}}$ and $\frac{\partial E_{\rm{xc}}}{\partial\rho}= V_{\rm{xc}}$.

In [31]:
D = np.array(wfn.Da())

V = np.zeros_like(D)
xc_e = 0.0

rho = []
points_func = Vpot.properties()[0]
superfunc = Vpot.functional()

# Loop over the blocks
for b in range(Vpot.nblocks()):
    
    # Obtain block information
    block = Vpot.get_block(b)
    points_func.compute_points(block)
    npoints = block.npoints()
    lpos = np.array(block.functions_local_to_global())
    
    
    # Obtain the grid weight
    w = np.array(block.w())

    # Compute phi!
    phi = np.array(points_func.basis_values()["PHI"])[:npoints, :lpos.shape[0]]
    
    # Build a local slice of D
    lD = D[(lpos[:, None], lpos)]
    
    # Copmute rho
    rho = 2.0 * np.einsum('pm,mn,pn->p', phi, lD, phi)

    inp = {}
    inp["RHO_A"] = psi4.core.Vector.from_array(rho)
    
    # Compute the kernel
    ret = superfunc.compute_functional(inp, -1)
    
    # Compute the XC energy
    vk = np.array(ret["V"])[:npoints]
    xc_e += np.einsum('a,a->', w, vk)
        
    # Compute the XC derivative.
    v_rho_a = np.array(ret["V_RHO_A"])[:npoints]
    Vtmp = np.einsum('pb,p,p,pa->ab', phi, v_rho_a, w, phi)

    # Add the temporary back to the larger array by indexing, ensure it is symmetric
    V[(lpos[:, None], lpos)] += 0.5 * (Vtmp + Vtmp.T)


print("XC Energy %16.8f" % xc_e)
print("V matrix:")
print(V)

print("\nMatches Psi4 V: %s"% np.allclose(V, wfn.Va()))

XC Energy      -1.01067075
V matrix:
[[-0.82622 -0.4433   0.       0.       0.     ]
 [-0.4433  -0.41091  0.       0.       0.     ]
 [ 0.       0.      -0.7327   0.       0.     ]
 [ 0.       0.       0.      -0.7327   0.     ]
 [ 0.       0.       0.       0.      -0.7327 ]]

Matches Psi4 V: True


Refs:
- Johnson, B. G.; Fisch M. J.; *J. Chem. Phys.*, **1994**, *100*, 7429